# Imports

In [1]:
from requests import get
from bs4 import BeautifulSoup
import os
from time import sleep
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import unicodedata
import json
from wordcloud import WordCloud
import numpy as np
import pprint as pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import scipy.stats as sp

# Read in the data

In [ ]:
df = pd.read_csv('trigger_warning_tweets.csv', index_col=0)
df.head()

### There are nulls
Drop rows with null values

In [3]:
df.isna().sum()

title                      0
tweet                      1
trigger_scene              0
cleaned_text               3
stemmed_text               3
lemmatized_text            3
lemmatized_no_stopwords    7
stemmed_no_stopwords       5
dtype: int64

In [6]:
df.dropna(inplace=True)
df.isna().sum()

title                      0
tweet                      0
trigger_scene              0
cleaned_text               0
stemmed_text               0
lemmatized_text            0
lemmatized_no_stopwords    0
stemmed_no_stopwords       0
dtype: int64

### Creating the matrix of tfidf values for bigrams

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(2,2))
X = tfidf.fit_transform(df.lemmatized_no_stopwords)
y = df.trigger_scene

In [10]:
pd.DataFrame(X.todense(), columns=tfidf.get_feature_names()).head(10)

,007 novel,00s version,010 recommend,02 second,03 2018,03 snes,0539 debbie,06 tony,08 back,0806 email,...,zone pretty,zone right,zone start,zooey deschanel,zoolander acenterforants,zootopia going,zorx gamepad,zu belegen,zu perk,zune glengarry
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Splitting into train-test set
Making sure that tweets for the same movie are in the same data set. This is important so we can tally up the classifications for a final prediction later.

In [35]:
pre_split = pd.DataFrame(X.todense(), columns=tfidf.get_feature_names()).join(pd.DataFrame(y)).join(df.title)
pre_split = pre_split.fillna(0)

no_scene_df = pre_split[pre_split.trigger_scene == False]

has_scene_df = pre_split[pre_split.trigger_scene == True]

2883
2883


In [36]:
n_self_harm = len(has_scene_df.title.unique())
n_no_self_harm = len(no_scene_df.title.unique())
percent_in_train = 0.7

print(f'Number of movies with self-harm scenes: {n_self_harm}')
print(f'Number of movies with no self-harm scenes: {n_no_self_harm}')

print('----------------------------------------------------------')

n_self_harm_in_train = round(n_self_harm * percent_in_train)
n_no_self_harm_in_train = round(n_no_self_harm * percent_in_train)

print(f'Number of self-harm movies to put into the train set: {n_self_harm_in_train}')
print(f'Number of no self-harm movies to put into the train set: {n_no_self_harm_in_train}')

Number of movies with self-harm scenes: 129
Number of movies with no self-harm scenes: 83
----------------------------------------------------------
Number of self-harm movies to put into the train set: 90
Number of no self-harm movies to put into the train set: 58


##### Train and test X sets

In [37]:
last_has_scene_movie_in_train = has_scene_df.title.unique()[:n_self_harm_in_train][-1]
index_of_last_has_scene_movie_in_train = (has_scene_df[has_scene_df.title == last_has_scene_movie_in_train]
                                          .index[-1])

has_scene_rows_to_include_in_train = has_scene_df.loc[:index_of_last_has_scene_movie_in_train].iloc[:,:-2]
has_scene_rows_to_include_in_test = has_scene_df.loc[index_of_last_has_scene_movie_in_train + 1:].iloc[:,:-2]

last_no_scene_movie_in_train = no_scene_df.title.unique()[:n_no_self_harm_in_train][-1]
index_of_last_no_scene_movie_in_train = (no_scene_df[no_scene_df.title == last_no_scene_movie_in_train]
                                          .index[-1])

no_scene_rows_to_include_in_train = no_scene_df.loc[:index_of_last_no_scene_movie_in_train].iloc[:,:-2]
no_scene_rows_to_include_in_test = no_scene_df.loc[index_of_last_no_scene_movie_in_train + 1:].iloc[:,:-2]

In [38]:
X_train = pd.concat([no_scene_rows_to_include_in_train, has_scene_rows_to_include_in_train])
X_test = pd.concat([no_scene_rows_to_include_in_test, has_scene_rows_to_include_in_test])

print(f'Number of rows in train: {len(X_train)}')
print(f'Number of rows in test: {len(X_test)}')

Number of rows in train: 2033
Number of rows in test: 850


In [39]:
if (len(X_train) + len(X_test)) == df.shape[0]:
    print('Number of rows match up')
else:
    print('Number of rows do not match up')

Number of rows match up


##### Train and test y sets

In [40]:
has_scene_class_rows_to_include_in_train = has_scene_df.loc[:index_of_last_has_scene_movie_in_train].iloc[:,-2]
has_scene_class_rows_to_include_in_test = has_scene_df.loc[index_of_last_has_scene_movie_in_train + 1:].iloc[:,-2]

no_scene_class_rows_to_include_in_train = no_scene_df.loc[:index_of_last_no_scene_movie_in_train].iloc[:,-2]
no_scene_class_rows_to_include_in_test = no_scene_df.loc[index_of_last_no_scene_movie_in_train + 1:].iloc[:,-2]

In [41]:
y_train = pd.concat([no_scene_class_rows_to_include_in_train, has_scene_class_rows_to_include_in_train])
y_test = pd.concat([no_scene_class_rows_to_include_in_test, has_scene_class_rows_to_include_in_test])

print(f'Number of rows in train: {len(y_train)}')
print(f'Number of rows in test: {len(y_test)}')

Number of rows in train: 2033
Number of rows in test: 850


In [61]:
y_train = y_train.astype('bool')
y_test = y_test.astype('bool')

In [62]:
if (len(y_train) + len(y_test)) == df.shape[0]:
    print('Number of rows match up')
else:
    print('Number of rows do not match up')

Number of rows match up


In [43]:
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

# Modeling
We will cross validate with RandomizedSearchCV first to determine hyperparameters.

### Logistic Regression

In [70]:
lr = LogisticRegression(random_state=123)
parameters = {'C':sp.reciprocal(0.0001, 10000),
              'solver':['newton-cg', 'lbfgs', 'saga', 'liblinear']}

lr_rs = RandomizedSearchCV(estimator=lr, param_distributions=parameters, n_jobs=4, random_state=123, n_iter=50)
lr_rs.fit(X_train, y_train)

C:\Users\mz211\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=4,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EBD77AAB38>, 'solver': ['newton-cg', 'lbfgs', 'saga', 'liblinear']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [71]:
print(lr_rs.best_params_)
print(lr_rs.best_score_)

{'C': 3.367745609488227, 'solver': 'liblinear'}
0.5927201180521396


### Using GridSearchCV to see if there's a difference

In [68]:
parameters = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'solver':['newton-cg', 'lbfgs', 'saga', 'liblinear']}

lr_gs = GridSearchCV(estimator=lr, param_grid=parameters, n_jobs=4)
lr_gs.fit(X_train, y_train)

C:\Users\mz211\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'solver': ['newton-cg', 'lbfgs', 'saga', 'liblinear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [69]:
print(lr_gs.best_params_)
print(lr_gs.best_score_)

{'C': 1, 'solver': 'newton-cg'}
0.5917363502213477


##### Looks like there isn't much of a difference in accuracy.
If anything, RandomizedSearchCV is providing slightly better hyperparameters in this case.

I'll continue with RandomizedSearch.

### Decision Tree

In [73]:
dt = DecisionTreeClassifier(random_state=123)
parameters = {'criterion':['gini', 'entropy'],
              'max_depth':[int(x) for x in np.linspace(3, 25, 3)],
              'min_samples_split':[int(x) for x in np.linspace(2,50, 2)],
              'min_samples_leaf':[1, 2, 3, 4, 5]}

dt_rs = RandomizedSearchCV(estimator=dt, param_distributions=parameters, n_jobs=4, n_iter=25)
dt_rs.fit(X_train, y_train)

C:\Users\mz211\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          fit_params=None, iid='warn', n_iter=25, n_jobs=4,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': [3, 14, 25], 'min_samples_split': [2, 50], 'min_samples_leaf': [1, 2, 3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [74]:
print(dt_rs.best_params_)
print(dt_rs.best_score_)

{'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 25, 'criterion': 'entropy'}
0.5602557796360059


### Random Forest

In [75]:
rf = RandomForestClassifier(random_state=123)
parameters = {'n_estimators':[int(x) for x in np.linspace(5, 50, 5)],
              'criterion':['gini', 'entropy'],
              'max_depth':[int(x) for x in np.linspace(3, 30, 3)],
              'min_samples_split':[int(x) for x in np.linspace(2, 20, 2)],
              'min_samples_leaf':[int(x) for x in np.linspace(1, 3, 1)]}

rf_rs = RandomizedSearchCV(estimator=rf, param_distributions=parameters, n_jobs=4, n_iter=50)
rf_rs.fit(X_train, y_train)

C:\Users\mz211\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=50, n_jobs=4,
          param_distributions={'n_estimators': [5, 16, 27, 38, 50], 'criterion': ['gini', 'entropy'], 'max_depth': [3, 16, 30], 'min_samples_split': [2, 20], 'min_samples_leaf': [1]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [76]:
print(rf_rs.best_params_)
print(rf_rs.best_score_)

{'n_estimators': 5, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30, 'criterion': 'entropy'}
0.5622233152975897


In [ ]:
knneighbors = KNeighborsClassifier()
parameters = {'n_neighbors':[3, 5, 7, 9, 13],
              'weights':['uniform', 'distance'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'p':[1, 2],
              'metric':['minkowski', 'euclidean', 'manhattan']}

knneighbors_rs = RandomizedSearchCV(estimator=knneighbors, param_distributions=parameters, n_jobs=4, n_iter=10, verbose=10)
knneighbors_rs.fit(X_train, y_train)

C:\Users\mz211\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
print(knneighbors_rs.best_params_)
print(knneighbors_rs.best_score_)

### XGBoost

In [ ]:
xgbc = xgb.sklearn.XGBClassifier()